Python libraries.

In [18]:
import praw
import pandas as pd
from nltk.tokenize import WhitespaceTokenizer
import re

Configure PRAW library connection to Reddit API and bind instance to "reddit" variable.

In [2]:
reddit = praw.Reddit(client_id = "OdtsOvr97jHWbkQt6_habg", #peronal use script ID
                    client_secret = "xBTlIHnYgXrANCTDLP0ZCDupPfUNbg", #secret token
                    usernme = "rock4777", #profile username
                    password = "geckoman1995", #profile password
                    user_agent = "USERAGENT")

Create empty lists to be filled with post data.

In [3]:
id_list = []
title_list = []
upvote_ratio_list = []
score_list = []

Loop through posts and fill lists.

In [4]:

for submission in reddit.subreddit("wallstreetbets").hot(limit = 10000):
    id_list.append(submission.id)
    title_list.append(submission.title)
    score_list.append(submission.score)
    upvote_ratio_list.append(submission.upvote_ratio)
    
print('total', len(title_list), 'posts has been scraped')

total 698 posts has been scraped


Create data frame by combining all lists.

In [5]:
df = pd.DataFrame({'ID':id_list, 
                   'Title':title_list,
                   'Upvote_Count':score_list,
                   'Upvote_Ratio':upvote_ratio_list,
                  })

Tokenize post titles.

In [13]:
tk = WhitespaceTokenizer()
token_list = []

for row in df.itertuples():
    tmp_list = tk.tokenize(row.Title)
    token_list.extend(tmp_list)

Remove all tokens that do not contain a "$".

In [14]:
token_list = list(filter(lambda k: '$' in k, token_list))

Function to check if a character is a number.

In [15]:
def containsNumber(value):
    for character in value:
        if character.isdigit():
            return True
    return False

Remove all rows containing numbers.

In [16]:
ticker_list = []

for token in token_list:
    if containsNumber(token) == False:
        ticker_list.append(token)
        

Upper case all tickers.

In [17]:
ticker_list = list(map(lambda x: x.upper(), ticker_list))

Remove all remaining special characters.

In [19]:
for i, ticker in enumerate(ticker_list):
    ticker_list[i] = re.sub('[^A-Za-z0-9]+', '', ticker)

Remove any empty rows.

In [20]:
ticker_list = list(filter(None, ticker_list))

Export as CSV.

In [24]:
export_df = pd.DataFrame(ticker_list) 
export_df.rename(columns = {0:'Ticker'}, inplace = True)
export_df.to_csv('ticker_list.csv')